In [1]:
import pandas as pd
import time
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import sys
import datetime
import os
import seaborn as sns
import math as mth
import pickle
import re
import nltk
import spacy
import sklearn

import tika
from tika import parser

import bokeh
from bokeh.plotting import *
from bokeh.io import output_notebook
from bokeh.models import Toggle, BoxAnnotation, CustomJS, HoverTool

output_notebook()

from sklearn.manifold import TSNE

sns.set_style("white")

%matplotlib inline

from IPython.display import display

from IPython.display import HTML

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Loading BokehJS ...

In [2]:
import pyodbc
from sqlalchemy import create_engine

engine = create_engine("your server engine here")

In [55]:
_author__ = 'zanderl'

from simple_salesforce import Salesforce
from sfdcSettings import *

#from FeedAdmin154.settings import SIMPLE_SALESFORCE_PASSWORD,SIMPLE_SALESFORCE_TOKEN,SIMPLE_SALESFORCE_USERNAME
# define your credentials here as variables inside this notebook or create a .py file containing them and import them to variable names

from operator import itemgetter
from datetime import datetime, timedelta
import pytz
import itertools as it

We'll use Pandas to read the SQL query so we can pull in the data directly from the database

The below query pulls transcripts for all of the companies within an individual or list of GICS industries

In [1]:
%%time

df1 = pd.read_sql_query('''

with test as (
select
row_number() over(partition by objectid order by transcriptcreationdateutc desc) as rownum,
evt.objectid,
tr.*
from ciqeventtoobjecttoeventtype evt
join ciqtranscript tr
on tr.keydevid = evt.keydevid
where objectid in(select 
companyid
from
ciqsubtypetogics gic
join ciqcompanyindustry ind
on ind.industryId = gic.subTypeId
where gic = '45102010')
)
Select * 
from 
test  ts
join ciqtranscriptcomponent ct
on ts.transcriptid = ct.transcriptid
where ts.transcriptid in (864627, 
7081, 
16557, 
708, 
4672, 
1001838, 
1001839, 
1001841, 
1001844, 
1002189
''', engine)

NameError: name 'pd' is not defined

In [23]:
df1.head()

,rownum,objectid,keyDevId,transcriptId,transcriptCollectionTypeId,transcriptPresentationTypeId,transcriptCreationDateUTC,audioLengthSec,transcriptComponentId,transcriptId,componentOrder,transcriptComponentTypeId,transcriptPersonId,componentText
0,1,24686,547249464,1377718,8,5,2018-02-01 05:05:26,3218,55062670,1377718,0,1,1,"Good day, ladies and gentlemen, and welcome to..."
1,1,24686,547249464,1377718,8,5,2018-02-01 05:05:26,3218,55062671,1377718,1,2,304268,"Thank you, Chelsea. Before we begin, I would l..."
2,1,24686,547249464,1377718,8,5,2018-02-01 05:05:26,3218,55062672,1377718,2,2,166284,"Thank you, Matt, and good afternoon to anyone ..."
3,1,24686,547249464,1377718,8,5,2018-02-01 05:05:26,3218,55062673,1377718,3,2,107251,"Thank you, Eli. First fiscal quarter revenue o..."
4,1,24686,547249464,1377718,8,5,2018-02-01 05:05:26,3218,55062674,1377718,4,7,1,[Operator Instructions] And our first question...


In [24]:
df1.describe()
df1.shape

,rownum,objectid,keyDevId,transcriptId,transcriptCollectionTypeId,transcriptPresentationTypeId,audioLengthSec,transcriptComponentId,transcriptId,componentOrder,transcriptComponentTypeId,transcriptPersonId
count,38.0,38.0,38.0,38.0,38.0,38.0,38.0,3.800000e+01,38.0,38.000000,38.000000,38.000000
mean,1.0,24686.0,547249464.0,1377718.0,8.0,5.0,3218.0,5.506269e+07,1377718.0,18.500000,4.105263,127053.473684
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.111306e+01,0.0,11.113055,1.673150,91156.488249
min,1.0,24686.0,547249464.0,1377718.0,8.0,5.0,3218.0,5.506267e+07,1377718.0,0.000000,1.000000,1.000000
25%,1.0,24686.0,547249464.0,1377718.0,8.0,5.0,3218.0,5.506268e+07,1377718.0,9.250000,3.000000,93296.000000
50%,1.0,24686.0,547249464.0,1377718.0,8.0,5.0,3218.0,5.506269e+07,1377718.0,18.500000,4.000000,136767.500000
75%,1.0,24686.0,547249464.0,1377718.0,8.0,5.0,3218.0,5.506270e+07,1377718.0,27.750000,4.000000,166284.000000
max,1.0,24686.0,547249464.0,1377718.0,8.0,5.0,3218.0,5.506271e+07,1377718.0,37.000000,7.000000,304268.000000


(38, 14)

In [25]:
df1['componentText']

0     Good day, ladies and gentlemen, and welcome to...
1     Thank you, Chelsea. Before we begin, I would l...
2     Thank you, Matt, and good afternoon to anyone ...
3     Thank you, Eli. First fiscal quarter revenue o...
4     [Operator Instructions] And our first question...
5     Yes. Matt Van Vliet, on for Tom. I guess, real...
6     So it's not only ONAP that is going to compens...
7     And then I guess as a follow-up to that, looki...
8     So we are not so sensitive to whether the indu...
9     And our next question comes from the line of J...
10    First question, on the Philippine customer tha...
11    So this is a managed transformation deal which...
12    Okay. Fair enough. And then, Tamar, there was ...
13    Yes. On overall basis for the year, we keep on...
14    And our next question comes from the line of A...
15    Question on Vubiquity. It seemed strategically...
16    So Ashwin, this is a scarce asset that we are ...
17    Great. And let me ask a second question. B

In [7]:
%%time

df3 = pd.read_sql_query('''

select 
*
from
ciqsubtypetogics gic
join ciqcompanyindustry ind
on ind.industryId = gic.subTypeId
where gic = '30101040'
''', engine)

Wall time: 247 ms


In [8]:
df3.shape
df3.head()

(69, 4)

,subTypeId,GIC,companyId,industryId
0,9600181,30101040,91638,9600181
1,9600181,30101040,92817,9600181
2,9600181,30101040,153820,9600181
3,9600181,30101040,262198,9600181
4,9600181,30101040,313055,9600181


Pull transcripts for all companies within a specified GICS grouping

In [9]:
%%time

df2 = pd.read_sql_query('''

with test as (
select
row_number() over(partition by objectid order by transcriptcreationdateutc desc) as rownum,
evt.objectid,
tr.*
from ciqeventtoobjecttoeventtype evt
join ciqtranscript tr
on tr.keydevid = evt.keydevid
where objectid in(select 
companyid
from
ciqsubtypetogics gic
join ciqcompanyindustry ind
on ind.industryId = gic.subTypeId
where gic = '30101040')
)
Select * 
from 
test  ts
join ciqtranscriptcomponent ct
on ts.transcriptid = ct.transcriptid
''', engine)

Wall time: 4min 39s


In [39]:
df2.shape
df2.head()

(103931, 14)

,rownum,objectid,keyDevId,transcriptId,transcriptCollectionTypeId,transcriptPresentationTypeId,transcriptCreationDateUTC,audioLengthSec,transcriptComponentId,transcriptId,componentOrder,transcriptComponentTypeId,transcriptPersonId,componentText
0,325,313055,3562009,9569,6,5,2008-06-19 23:01:30,NaN,79109,9569,1,5,NaN,This call is the property of Wal-Mart Stores I...
1,326,313055,3562010,9570,6,5,2008-06-19 23:01:30,NaN,79110,9570,1,5,NaN,Thank you for calling Wal-Mart Stores Incorpor...
2,327,313055,3562011,9571,6,5,2008-06-19 23:01:30,NaN,79111,9571,1,5,NaN,Thank you for calling Wal-Mart Stores Incorpor...
3,328,313055,5169974,9572,6,5,2008-06-19 23:01:30,NaN,79112,9572,1,5,NaN,This call is the property of Wal-Mart Stores I...
4,329,313055,5169797,9574,6,5,2008-06-19 23:01:30,NaN,79113,9574,1,5,NaN,"Thank you for calling Wal-Mart Stores, Inc. Fi..."


In [10]:
df2['componentText']

0         This call is the property of Wal-Mart Stores I...
1         Thank you for calling Wal-Mart Stores Incorpor...
2         Thank you for calling Wal-Mart Stores Incorpor...
3         This call is the property of Wal-Mart Stores I...
4         Thank you for calling Wal-Mart Stores, Inc. Fi...
5         Good morning everyone and welcome to the BJ's ...
6         Thank you, Jim. With me this morning are Herb ...
7         Thank you, Cathy. Good morning everyone. For t...
8         Thank you, Frank, and good morning everyone. T...
9         Thanks Herb. For the benefit of your P&L model...
10        An awful lot of information, Frank. Thanks. We...
11        Thank you, gentlemen. [Operator Instructions]....
12                                    Hi, good morning.\r\n
13                                        Good morning.\r\n
14        Two questions. The first one is for Frank. On ...
15        The discontinued operations, if that's your qu...
16        Okay. And on the -- as you loo

In [11]:
import spacy
nlp = spacy.load('en')

In [12]:
TScomponents = pd.Series(df2['componentText'])
TScomponents.shape
TScomponents[0]

(103931,)

"This call is the property of Wal-Mart Stores Inc. and is intended solely for use of Wal-Mart shareholders. It should not be reproduced in any way.\r\nThis call will contain statements that Wal-Mart believes are forward-looking statements within the meaning of the Private Securities Litigation Reform Act of 1995 and intended to enjoy the protection of Safe Harbor for forward-looking statements provided by that Act. These forward-looking statements generally are identified by the use of the word or phrases: expect, invest, may impact, opening, plan, will be, will continue, will enhance, will have, will reflect, and will take or a variation on one of those words or phrases in those statements, or by the use of words or phrases of similar import. Similarly, descriptions of our objectives, plans, goals, targets, or expectations are forward-looking statements.\r\nThese statements discuss, among other things, our anticipated U.S. comparable store sales for the current fiscal quarter and our 

In [13]:
TScomponents.describe()

count     103931
unique     61586
top         Yes.
freq         676
Name: componentText, dtype: object

In [25]:
processed_text = nlp(TScomponents[0])

In [70]:
# %%time
# for i in TScomponents:
#     processed_text = nlp(i)
# #processed_text

Wall time: 1h 14min 33s


In [26]:
for num, entity in enumerate(processed_text.ents):
    'Entity {}:'.format(num + 1), entity, entity.label_

('Entity 1:', Wal-Mart Stores Inc., 'ORG')

('Entity 2:', Wal-Mart, 'ORG')

('Entity 3:', Wal-Mart, 'ORG')

('Entity 4:', the Private Securities Litigation Reform Act of 1995, 'ORG')

('Entity 5:', Safe Harbor, 'FAC')

...

('Entity 524:', the second quarter, 'DATE')

('Entity 525:', between $0.75, 'MONEY')

('Entity 526:', 0.79, 'MONEY')

('Entity 527:', Wal-Mart, 'ORG')

('Entity 528:', 
 , 'PERSON')

In [27]:
token_text =[token.orth_ for token in processed_text]
token_pos = [token.pos_ for token in processed_text]
token_lemma = [token.lemma_ for token in processed_text]
token_shape = [token.shape_ for token in processed_text]
token_entity_type = [token.ent_type_ for token in processed_text]
token_entity_iob = [token.ent_iob_ for token in processed_text]


text_df1 = pd.DataFrame(token_text, columns=['token_text'])
text_df1['part_of_speech'] = token_pos
text_df1['token_lemma'] = token_lemma
text_df1['token_shape'] = token_shape
text_df1['entity_type'] = token_entity_type
text_df1['token_inside_outside_begin'] = token_entity_iob
text_df1.head()

,token_text,part_of_speech,token_lemma,token_shape,entity_type,token_inside_outside_begin
0,This,DET,this,Xxxx,,O
1,call,NOUN,call,xxxx,,O
2,is,VERB,be,xx,,O
3,the,DET,the,xxx,,O
4,property,NOUN,property,xxxx,,O


In [28]:
text_df1.shape

(6904, 6)

In [39]:
#intermediate_directory = os.path.join('..','intermediate')
#ts_text_filepath = os.path.join(intermediate_directory, 'ts_text_all.txt')

In [44]:
TScomponents_all = TScomponents.str.cat(sep=' ')

In [47]:
len(TScomponents_all)

98593423

In [48]:
import os
import codecs

file_path = 'C:/Users/zanderl/TS_text.txt'

with codecs.open(file_path,'w',encoding='utf_8') as ts_text_file:
    ts_text_file.write(TScomponents_all.replace('\n','\\n') + '\n')

In [49]:
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

In [52]:
def punct_space(token):
    
    """Helper function to eliminate tokens that are pure punctuation or whitespace"""
    
    return token.is_punct or token.is_space

def line_transcript(filename):
    """
    Generator function to read in transcript data from the file and un-escape the orignial line breaks in the text
    """
    
    with codecs.open(filename, encoding='utf-8') as f:
        for transcript in f:
            yield transcript.replace('\\n', '\n')
            
def lemmatized_sentence_corpus(filename):
    
    '''
    Helper function to parse through transcripts using SpaCy
    Lemmatizes the text, and then yields sentences
    '''
    
    for transcript in nlp.pipe(line_transcript(filename),
                               batch_size=10000,n_threads=4):
        
        for sent in transcript.sents:
            yield u' '.join([token.lemma_ for token in sent
                            if not punct_space(token)])

In [53]:
%%time

with codecs.open("unigram_sentences.txt",'w',encoding='utf_8') as f:
    for sentence in lemmatized_sentence_corpus('TS_text.txt'):
        f.write(sentence + '\n')

Wall time: 1h 13min 58s


In [54]:
unigram_sentences = LineSentence('unigram_sentences.txt')

In [62]:
for unigram_sentence in it.islice(unigram_sentences, 10, 20):
    print(u' '.join(unigram_sentence))
    print(u' ')

the forward look statement make in this call be make on and as of the date of this call and -PRON- undertake no obligation to update these forward look statement to reflect subsequent event or circumstance
 
the comp store sale for -PRON- total u.s. operation and for -PRON- sam 's club segment discuss on this call exclude the impact of fuel sale at -PRON- sam 's club segment
 
that measure and -PRON- return on investment as discuss in this call may be consider non gaap financial measure
 
reconciliation of those non gaap financial measure to the most directly comparable gaap measure be available for review on the investor relations portion of -PRON- corporate website at www.walmartstores.com/investor
 
< strong > carol schumacher < /strong
 
>
 
welcome to the wal mart stores inc. first quarter earning call for fiscal year 2008
 
this be carol schumacher vice president of investor relations
 
the replay of this call and related material about this quarter be available on -PRON- website